In [ ]:
/resource/35985678-0d79-46b4-9ed6-6f13308a1d24

In [ ]:
Mandi Data api

https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001e3a48235641a458b502501e95c36f78e&format=json

In [ ]:
GNews API 
https://gnews.io/api/v4/{endpoint}?{parameters}&apikey=698746a0423c3a255985f83db1100e7c


In [ ]:
Weather =bit API

https://api.weatherbit.io/v2.0/history/subhourly?lat=35.78&lon=78.64&start_date=2025-11-07&end_date=2025-11-08&key=ea173c6e1835424e9668ea1d8a7a4b9b



In [13]:
import pandas as pd
import re
import requests
import os
import time

# --- 1. Load and Parse the Market Excel File ---

def parse_market_file(file_name):
    """
    Reads the Excel file and extracts market coordinates
    from the first column using regular expressions.
    """
    market_data = []
    # Regex to find "MarketName": (Lat, Lon)
    pattern = re.compile(r'"([^"]+)":\s*\(\s*([0-9.-]+)\s*,\s*([0-9.-]+)\s*\)')
    
    try:
        # Use read_excel for .xlsx files
        df = pd.read_excel(file_name, engine='openpyxl')
        
        if df.empty:
            print("The market file is empty.")
            return None
            
        column_name = df.columns[0]
        
        for line in df[column_name]:
            if not isinstance(line, str):
                continue
                
            match = pattern.search(line)
            if match:
                market_data.append({
                    "Market_Name": match.group(1),
                    "Latitude": float(match.group(2)),
                    "Longitude": float(match.group(3))
                })
                
        if not market_data:
            print("Could not parse any market data from the file.")
            return None
            
        print(f"Successfully parsed {len(market_data)} markets from Excel file.")
        return pd.DataFrame(market_data)
        
    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")
        return None
    except ImportError:
        print("Error: The 'openpyxl' library is required to read .xlsx files.")
        print("Please install it: pip install openpyxl")
        return None
    except Exception as e:
        print(f"An error occurred parsing the file: {e}")
        return None

# --- 2. Fetch Weather Data from Open-Meteo API ---

def fetch_weather_for_markets(df_markets):
    """
    Fetches DAILY historical weather data from the Open-Meteo API
    for the specified date range.
    
    This API is free, requires no key, and can handle all locations
    and the full date range in a single API call.
    """
    
    # Open-Meteo Historical (Archive) API endpoint
    BASE_URL = "https://archive-api.open-Meteo.com/v1/archive"
    
    # Your requested date range
    START_DATE = "2024-01-01"
    END_DATE = "2025-01-01"
    
    # Define the daily weather variables we want.
    # See docs for more: https://open-meteo.com/en/docs/historical-weather-api
    DAILY_VARIABLES = [
        "temperature_2m_max",
        "temperature_2m_min",
        "temperature_2m_mean",
        "precipitation_sum",
        "rain_sum",
        "snowfall_sum",
        "wind_speed_10m_max",
        "wind_gusts_10m_max",
        "weather_code",         # <-- ADDED
        "cloud_cover_mean",     # <-- ADDED
        "uv_index_max"          # <-- ADDED
    ]
    
    print(f"\nStarting API call for {len(df_markets)} markets (using Open-Meteo)...")
    print(f"Querying URL: {BASE_URL}")
    print(f"Querying Dates: {START_DATE} to {END_DATE}")

    # --- Build one API call for ALL markets ---
    # Open-Meteo is powerful and can take arrays of lat/lon
    
    all_lats = df_markets['Latitude'].tolist()
    all_lons = df_markets['Longitude'].tolist()
    all_market_names = df_markets['Market_Name'].tolist()

    params = {
        'latitude': all_lats,
        'longitude': all_lons,
        'start_date': START_DATE,
        'end_date': END_DATE,
        'daily': DAILY_VARIABLES,
        'timezone': 'auto' # Automatically adjust to the local timezone
    }

    all_market_dataframes = [] # List to hold final dataframes
    
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # Raise error for bad responses
        
        data = response.json()
        
        if not isinstance(data, list):
            # If only one location, API returns a single object. Wrap it in a list.
            data = [data]

        print(f"  > Successfully fetched data for {len(data)} locations in one call.")

        # --- Process the response ---
        # `data` is a list, each item corresponds to one location
        for i, location_data in enumerate(data):
            market_name = all_market_names[i]
            lat = all_lats[i]
            lon = all_lons[i]
            
            daily_data = location_data.get('daily')
            if not daily_data:
                print(f"  > No 'daily' data returned for: {market_name}")
                continue

            df_daily = pd.DataFrame(daily_data)
            
            # Add the market context to this dataframe
            df_daily['market_name_lookup'] = market_name
            df_daily['latitude_lookup'] = lat
            df_daily['longitude_lookup'] = lon
            
            all_market_dataframes.append(df_daily)
            print(f"  > Processed data for: {market_name}")

    except requests.exceptions.HTTPError as http_err:
        print(f"  > FAILED (HTTP Error): {http_err}")
    except requests.exceptions.RequestException as e:
        print(f"  > FAILED (Connection Error): {e}")
    except Exception as e:
        print(f"  > FAILED (Processing Error): {e}")

    if not all_market_dataframes:
        print("\nNo weather data was collected from the API.")
        return None
        
    # Combine all individual market dataframes into one large dataframe
    final_df = pd.concat(all_market_dataframes, ignore_index=True)
    return final_df

# --- 3. Main execution ---

def main():
    input_file = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Market.xlsx"
    # Output file name
    output_file = "market_historical_weather_daily.xlsx"
    
    df_markets = parse_market_file(input_file)
    
    if df_markets is not None:
        df_weather = fetch_weather_for_markets(df_markets)
        
        if df_weather is not None:
            # We have one big DataFrame. We can still save it to one
            # Excel file, but saving each market to its own sheet
            # is still a good idea for readability.
            
            print(f"\nWriting all market data to Excel file: {output_file}")
            
            try:
                with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                    # Group the big DataFrame by market name
                    grouped = df_weather.groupby('market_name_lookup')
                    
                    for market_name, df_market_data in grouped:
                        # Clean sheet name (Excel limits to 31 chars, no special chars)
                        safe_sheet_name = re.sub(r'[\/:*?\[\]]', '', market_name)[:31]
                        
                        # --- Re-order columns for better readability ---
                        # We define the desired order here
                        cols_to_order = [
                            'market_name_lookup', 
                            'latitude_lookup', 
                            'longitude_lookup', 
                            'time', 
                            'temperature_2m_max', 
                            'temperature_2m_min', 
                            'temperature_2m_mean',
                            'precipitation_sum', 
                            'rain_sum', 
                            'snowfall_sum',
                            'wind_speed_10m_max', 
                            'wind_gusts_10m_max',
                            'weather_code',
                            'cloud_cover_mean',
                            'uv_index_max'
                        ]
                        
                        # Get a list of columns that actually exist in the dataframe
                        # in the desired order
                        existing_cols_in_order = [c for c in cols_to_order if c in df_market_data.columns]
                        
                        # Get any other columns that might exist but aren't in our list
                        # (so we don't lose data)
                        other_cols = [c for c in df_market_data.columns if c not in existing_cols_in_order]
                        
                        # Final column list
                        final_cols = existing_cols_in_order + other_cols
                        
                        print(f"  > Writing sheet: {safe_sheet_name}...")
                        # Save to excel with the new column order
                        df_market_data[final_cols].to_excel(writer, sheet_name=safe_sheet_name, index=False)
                
                print(f"\nSuccessfully collected all data and saved to '{output_file}'.")
            
            except ImportError:
                print("\nError: 'openpyxl' is required to write Excel files.")
                print("Please install it: pip install openpyxl")
            except Exception as e:
                print(f"\nAn error occurred while writing the Excel file: {e}")
                
        else:
            print("Failed to fetch any weather data.")

if __name__ == "__main__":
    main()

Successfully parsed 21 markets from Excel file.

Starting API call for 21 markets (using Open-Meteo)...
Querying URL: https://archive-api.open-Meteo.com/v1/archive
Querying Dates: 2024-01-01 to 2025-01-01
  > Successfully fetched data for 21 locations in one call.
  > Processed data for: Bardoli
  > Processed data for: Bardoli(Kat)
  > Processed data for: Bardoli(Ma)
  > Processed data for: Kosamba
  > Processed data for: Kosamba(D)
  > Processed data for: Mahuva
  > Processed data for: Mahuva(Am)
  > Processed data for: Mandvi
  > Processed data for: Nizar
  > Processed data for: Nizar(Kuka)
  > Processed data for: Nizar(Pumb)
  > Processed data for: S.Mandvi
  > Processed data for: Songadh
  > Processed data for: Songadh(B)
  > Processed data for: Songadh(U)
  > Processed data for: Surat
  > Processed data for: Uchhal
  > Processed data for: Valod
  > Processed data for: Valod(Buh)
  > Processed data for: Vyara(Pan)
  > Processed data for: Vyra

Writing all market data to Excel file:

In [1]:
import pandas as pd
import os

# Define the path to the Excel file
excel_path = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\market_historical_weather_daily.xlsx"

# Define the output directory for CSV files
output_dir = os.path.join(os.path.dirname(excel_path), "csv_exports")
os.makedirs(output_dir, exist_ok=True)

# Load the Excel file
excel_file = pd.ExcelFile(excel_path)

# Loop through each sheet and export to CSV
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    csv_filename = f"{sheet_name}.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    df.to_csv(csv_path, index=False)
    print(f"Exported: {csv_path}")


Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Bardoli.csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Bardoli(Kat).csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Bardoli(Ma).csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Kosamba.csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Kosamba(D).csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Mahuva.csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Mahuva(Am).csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Mandvi.csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Nizar.csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Nizar(Kuka).csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\Nizar(Pumb).csv
Exported: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports\S.Mandvi.csv
Exported

In [5]:
import mysql.connector

# Connect to your MySQL server
conn = mysql.connector.connect(
    host="localhost",         # or your server IP
    user="root",
    password="root",
    database="weather_history"
)
cursor = conn.cursor()

# List of cleaned market names
market_names = [
    "Bardoli_Kar", "Bardoli_Men", "Kosamba", "Kosamba_D", "Mahuva",
    "Mahuva_Am", "Mandvi", "Nizar", "Nizar_Kuka", "Nizar_Purn", "S_Mandvi",
    "Songadh", "Songadh_B", "Songadh_U", "Surat", "Uchhal", "Valod",
    "Valod_Buh", "Vyara_Pan", "Vyra"
]

# SQL template
table_schema = """
DROP TABLE IF EXISTS {table_name};

CREATE TABLE {table_name} (
    market_name VARCHAR(100),
    latitude DECIMAL(8,5),
    longitude DECIMAL(8,5),
    time DATE,
    temperature_max DECIMAL(5,2),
    temperature_min DECIMAL(5,2),
    temperature_mean DECIMAL(5,2),
    precipitation_sum DECIMAL(6,2),
    rain_sum DECIMAL(6,2),
    wind_speed_max DECIMAL(5,2),
    wind_gusts_max DECIMAL(5,2),
    weather_code INT,
    cloud_cover DECIMAL(5,2)
);
"""

# Execute SQL for each table
for name in market_names:
    clean_name = name.replace("(", "").replace(")", "").replace(".", "").replace(" ", "_").replace("-", "_")
    sql = table_schema.format(table_name=clean_name)
    for statement in sql.strip().split(";"):
        if statement.strip():
            cursor.execute(statement)

conn.commit()
cursor.close()
conn.close()
print("All tables created successfully.")


All tables created successfully.


Load CSVs into MySQL Tables

In [12]:
import os
import pandas as pd
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="weather_history"
)
cursor = conn.cursor()

# Folder containing CSV files
csv_folder = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\csv_exports"

# Expected column mapping (CSV → SQL)
column_map = {
    "market_name_lookup": "market_name",
    "latitude_lookup": "latitude",
    "longitude_lookup": "longitude",
    "temperature_2m_max": "temperature_max",
    "temperature_2m_min": "temperature_min",
    "temperature_2m_mean": "temperature_mean",
    "precipitation_sum": "precipitation_sum",
    "rain_sum": "rain_sum",
    "wind_speed_10m_max": "wind_speed_max",
    "wind_gusts_10m_max": "wind_gusts_max",
    "weather_code": "weather_code",
    "cloud_cover_mean": "cloud_cover",
    "time": "time"
}

# Table names
market_names = [
    "Bardoli", "Bardoli_Kar", "Bardoli_Ma", "Kosamba", "Kosamba_D", "Mahuva",
    "Mahuva_Am", "Mandvi", "Nizar", "Nizar_Kuka", "Nizar_Pumb", "S_Mandvi",
    "Songadh", "Songadh_B", "Songadh_U", "Surat", "Uchhal", "Valod",
    "Valod_Buh", "Vyara_Pan", "Vyra"
]

# Mapping for mismatched CSV filenames
csv_name_map = {
    "Bardoli_Kar": "Bardoli(Kat)",
    "Nizar_Kuka": "Nizar(Kuka)",
    "S_Mandvi": "S.Mandvi",
    "Songadh_B": "Songadh(B)",
    "Songadh_U": "Songadh(U)",
    "Bardoli_Ma": "Bardoli(Ma)",
    "Kosamba_D": "Kosamba(D)",
    "Mahuva_Am": "Mahuva(Am)",
    "Nizar_Pumb": "Nizar(Pumb)",
    "Valod_Buh": "Valod(Buh)",
    "Vyara_Pan": "Vyara(Pan)"

}

# Load and insert each CSV
for table_name in market_names:
    # Determine CSV filename
    csv_filename = csv_name_map.get(table_name, table_name) + ".csv"
    csv_path = os.path.join(csv_folder, csv_filename)

    if not os.path.exists(csv_path):
        print(f"❌ CSV not found: {csv_path}")
        continue

    # Check if table already has data
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    if row_count > 0:
        print(f"⏭️ Skipping {table_name}: already contains {row_count} rows.")
        continue

    # Load CSV
    df = pd.read_csv(csv_path)

    # Rename columns to match SQL schema
    df.rename(columns=column_map, inplace=True)

    # Filter only expected columns
    expected_cols = list(column_map.values())
    df = df[[col for col in expected_cols if col in df.columns]]

    # Prepare insert query
    placeholders = ", ".join(["%s"] * len(df.columns))
    columns_str = ", ".join(df.columns)
    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"

    # Insert rows
    for row in df.itertuples(index=False):
        cursor.execute(insert_query, tuple(row))

    print(f"✅ Inserted data into table: {table_name}")

conn.commit()
cursor.close()
conn.close()
print("🎯 All CSVs processed.")


⏭️ Skipping Bardoli: already contains 367 rows.
⏭️ Skipping Bardoli_Kar: already contains 367 rows.
⏭️ Skipping Bardoli_Ma: already contains 367 rows.
⏭️ Skipping Kosamba: already contains 367 rows.
⏭️ Skipping Kosamba_D: already contains 367 rows.
⏭️ Skipping Mahuva: already contains 367 rows.
⏭️ Skipping Mahuva_Am: already contains 367 rows.
⏭️ Skipping Mandvi: already contains 367 rows.
⏭️ Skipping Nizar: already contains 367 rows.
⏭️ Skipping Nizar_Kuka: already contains 367 rows.
⏭️ Skipping Nizar_Pumb: already contains 367 rows.
⏭️ Skipping S_Mandvi: already contains 367 rows.
⏭️ Skipping Songadh: already contains 367 rows.
⏭️ Skipping Songadh_B: already contains 367 rows.
⏭️ Skipping Songadh_U: already contains 367 rows.
⏭️ Skipping Surat: already contains 367 rows.
⏭️ Skipping Uchhal: already contains 367 rows.
⏭️ Skipping Valod: already contains 367 rows.
⏭️ Skipping Valod_Buh: already contains 367 rows.
⏭️ Skipping Vyara_Pan: already contains 367 rows.
⏭️ Skipping Vyra: alrea

In [13]:
import pandas as pd

# Path to your CSV file
csv_path = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Agmarknet_Price_Report.csv"

# Load the CSV file
df = pd.read_csv(csv_path)

# Ensure the date column is parsed correctly
# Replace 'Arrival_Date' with the actual column name in your CSV that contains dates
df['Arrival_Date'] = pd.to_datetime(df['Arrival_Date'], dayfirst=True, errors='coerce')

# Define date range
start_date = pd.to_datetime("2024-01-01")
end_date = pd.to_datetime("2025-01-01")

# Filter rows within the date range
filtered_df = df[(df['Arrival_Date'] >= start_date) & (df['Arrival_Date'] <= end_date)]

# Save the filtered data to a new CSV
output_path = r"D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Agmarknet_Price_Report_2024.csv"
filtered_df.to_csv(output_path, index=False)

print(f"✅ Extracted {len(filtered_df)} records between {start_date.date()} and {end_date.date()}")
print(f"Saved to: {output_path}")


✅ Extracted 14965 records between 2024-01-01 and 2025-01-01
Saved to: D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Agmarknet_Price_Report_2024.csv
